In [ ]:
!conda install

# Assignment 3
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. All questions are worth the same number of points except question 1 which is worth 17% of the assignment grade.

**Note**: Questions 3-13 rely on your question 1 answer.

In [108]:
import pandas as pd
import numpy as np
import re
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('float_format', '{:f}'.format)
e = [str(x) for x in range(1960,2006)]
def dstrip(df,x):
    d = list(df[x])
    for i in d:
        r = d.index(i)
        d[r] = int(i)
def line_remove(df,x):
    d = list(df[x])
    for i in d:
        j = re.sub("[\(\[].*?[\)\]]", "", i)
        r = d.index(i)
        d[r] = j.strip()
    return d
def line_strip(df,x):
    d = list(df[x])
    for i in reversed(range(1,30)):
        y = str(i)
        for j in d:
            if j.endswith(y):
                r = d.index(j)
                d[r] = j.rstrip(y)
    return d
def unique(a,b,c,d):
    count = 0
    for i in a:
        if i in b:
            if i in c:
                if i in d:
                    b.remove(i)
                    c.remove(i)
                    d.remove(i)
                else:
                    continue
            else:
                continue
        else:
            continue
    return b,c,d
# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.  `'Switzerland17'` should be `'Switzerland'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [109]:
def answer_one():
    df = pd.read_excel("assets/Energy Indicators.xls",skiprows=9, skipfooter=38)
    df.drop(df.columns[[0, 1,]], axis=1, inplace =True)
    df = df.iloc[8:].set_index(df.index[:-8])
    df = df.rename(columns={"Energy Supply per capita": "Energy Supply per Capita", "Renewable Electricity Production": "% Renewable"})
    df.loc[df['Energy Supply'] == "...", 'Energy Supply']= np.nan
    df.loc[df['Energy Supply per Capita'] == "...", 'Energy Supply per Capita']= np.nan
    df['Energy Supply'] = df['Energy Supply'].mul(1000000)
    df['Country'] = line_strip(df,'Country')
    df['Country'] = line_remove(df,'Country')
    df.loc[df['Country'] == "Republic of Korea", 'Country']= "South Korea"
    df.loc[df['Country'] == "United States of America", 'Country']= "United States"
    df.loc[df['Country'] == "United Kingdom of Great Britain and Northern Ireland", 'Country']= "United Kingdom"
    df.loc[df['Country'] == "China, Hong Kong Special Administrative Region", 'Country']= "Hong Kong"
    df.loc[df['Country'] == "Iran, Islamic Rep.", 'Country']= "Iran"
    #df['Energy Supply per Capita']= dstrip(df,'Energy Supply per Capita')
    df1 = pd.read_csv("assets/world_bank.csv",skiprows=4, skipfooter=0)
    df1.rename({'Country Name':'Country'},axis='columns',inplace = True)
    df1.loc[df1['Country'] == "Korea, Rep.", 'Country']= "South Korea"
    df1.loc[df1['Country'] == "Iran, Islamic Rep.", 'Country']= "Iran"
    df1.loc[df1['Country'] == "Hong Kong SAR, China", 'Country']= "Hong Kong" 
    df2 = pd.read_excel("assets/scimagojr-3.xlsx")
    #raise NotImplementedError()
    dfs = df.merge( df1, how='inner',on='Country')
    dfs2 = pd.merge(dfs, df2, how='inner',on='Country')
    dfs2.drop(['Country Code', 'Indicator Name', 'Indicator Code'], axis = 1,inplace= True)
    dfs2= dfs2.sort_values('Rank').set_index('Country')
    column_names = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    dfs2 = dfs2.reindex(columns=column_names)
    df_sub = dfs2.head(15)
    return df_sub
answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.700000,138,127191000000,93,19.754910,3992331462570.890137,4559041383517.929688,4997775145605.959961,5459247267275.139648,6039658508485.589844,6612490294273.650391,7124977970399.610352,7672447738017.419922,8230120617793.519531,8797998940421.279297
United States,2,96661,94747,792274,265436,8.200000,230,90838000000,286,11.570980,14792303791800.000000,15055395304800.000000,15011490541400.000000,14594842181900.000000,14964372000000.000000,15204019634600.000000,15542161722300.000000,15773666098800.000000,16156618476000.000000,16548573278300.000000
Japan,3,30504,30287,223024,61554,7.310000,134,18984000000,149,10.232820,5496541517363.309570,5617035943770.730469,5558526873313.820312,5251308399909.320312,5498717815809.769531,5473738114222.049805,5569101602926.179688,5644658523313.419922,5642884448582.190430,5669563143140.969727
United Kingdom,4,20944,20357,206091,37874,9.840000,139,7920000000,124,10.600470,2419630700401.729980,2482203235475.899902,2470614405284.299805,2367047611711.990234,2403504326328.799805,2450911011124.850098,2479808627472.189941,2533370110012.359863,2605643189894.930176,2666333396477.129883
Russian Federation,5,18534,18301,34266,12422,1.850000,57,30709000000,214,17.288680,1385792607182.580078,1504071117741.300049,1583004071092.110107,1459199142720.260010,1524917468442.010010,1589942641764.290039,1645875899596.270020,1666934123163.159912,1678708855070.860107,1616148858033.770020
Canada,6,17899,17620,215003,40930,12.010000,149,10431000000,296,61.945430,1564468853329.449951,1596739898466.320068,1612713061250.239990,1565144676082.310059,1613406134731.120117,1664086754804.889893,1693132927878.080078,1730687770724.129883,1773485821199.770020,1792608764608.810059
Germany,7,17027,16831,140566,27426,8.260000,126,13261000000,165,17.901530,3332890754966.890137,3441560834437.089844,3478809377483.439941,3283339933774.830078,3417298013245.029785,3542371125827.810059,3556723774834.439941,3567317390728.479980,3624386278145.700195,3685555907284.770020
India,8,15005,14841,128763,37209,8.580000,115,33195000000,26,14.969080,1265894322131.090088,1374865195244.500000,1428360609657.800049,1549482541544.699951,1708458876829.919922,1821872415623.620117,1924235460945.250000,2051981849791.689941,2200616575310.970215,2367206118959.060059
France,9,13153,12973,130632,28601,9.930000,114,10597000000,166,17.020280,2607839735099.339844,2669423841059.600098,2674637086092.709961,2595966887417.220215,2646994701986.750000,2702031788079.470215,2706968211920.529785,2722566887417.220215,2729631788079.470215,2761185430463.580078


In [110]:
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"

assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"


In [111]:
# Cell for autograder.


### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [112]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [113]:
def final_others():
    df = pd.read_excel("assets/Energy Indicators.xls",skiprows=9, skipfooter=38)
    df.drop(df.columns[[0, 1,]], axis=1, inplace =True)
    df = df.iloc[8:].set_index(df.index[:-8])
    df = df.rename(columns={"Energy Supply per capita": "Energy Supply per Capita", "Renewable Electricity Production": "% Renewable"})
    df.loc[df['Energy Supply'] == "...", 'Energy Supply']= np.nan
    df.loc[df['Energy Supply per Capita'] == "...", 'Energy Supply per Capita']= np.nan
    df['Energy Supply'] = df['Energy Supply'].mul(1000000)
    df['Country'] = line_strip(df,'Country')
    df['Country'] = line_remove(df,'Country')
    df.loc[df['Country'] == "Republic of Korea", 'Country']= "South Korea"
    df.loc[df['Country'] == "United States of America", 'Country']= "United States"
    df.loc[df['Country'] == "United Kingdom of Great Britain and Northern Ireland", 'Country']= "United Kingdom"
    df.loc[df['Country'] == "China, Hong Kong Special Administrative Region", 'Country']= "Hong Kong"
    df.loc[df['Country'] == "Iran, Islamic Rep.", 'Country']= "Iran"
    df1 = pd.read_csv("assets/world_bank.csv",skiprows=4, skipfooter=0)
    df1.rename({'Country Name':'Country'},axis='columns',inplace = True)
    df1.loc[df1['Country'] == "Korea, Rep.", 'Country']= "South Korea"
    df1.loc[df1['Country'] == "Iran, Islamic Rep.", 'Country']= "Iran"
    df1.loc[df1['Country'] == "Hong Kong SAR, China", 'Country']= "Hong Kong" 
    df2 = pd.read_excel("assets/scimagojr-3.xlsx")
    #raise NotImplementedError()
    dfs = pd.merge(df, df2, how='inner',on='Country')
    dfs2 = pd.merge(dfs, df1, how='inner',on='Country')
    dfs2.drop(['Country Code', 'Indicator Name', 'Indicator Code'], axis = 1,inplace= True)
    dfs2= dfs2.sort_values('Rank').set_index('Country')
    column_names = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    dfs2 = dfs2.reindex(columns=column_names)
    return dfs2,df,df1,df2

In [114]:
def answer_two():
    dfs2,df,df1,df2 = final_others()
    dfs = pd.merge(df, df2, how='outer',on='Country')
    dfs3 = pd.merge(dfs, df1, how='outer',on='Country')
    return len(dfs3)-len(dfs2)
answer_two()

156

In [115]:
assert type(answer_two()) == int, "Q2: You should return an int number!"


### Question 3
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [272]:
def answer_three():
    from statistics import mean,stdev
    t = answer_one()
    #raise NotImplementedError()
    frames = ['2006','2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    t['AVG'] = t[frames].apply(lambda z: np.mean(z), axis=1)
    ts = pd.Series(t['AVG'].values, index=t.index)
    return ts.sort_values(ascending=False).head(15)
answer_three()

Country
United States        15364344302990.000000
China                 6348608932836.099609
Japan                 5542207638235.175781
Germany               3493025339072.847656
France                2681724635761.588867
United Kingdom        2487906661418.417480
Brazil                2189794143774.905029
Italy                 2120175089933.776123
India                 1769297396603.860107
Canada                1660647466307.511963
Russian Federation    1565459478480.660889
Spain                 1418078278145.694092
Australia             1164042729991.427002
South Korea           1106714508244.852051
Iran                   444155754051.094971
dtype: float64

In [117]:
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"


### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [275]:
def answer_four():
    t = answer_one()
    return t['2015']['United Kingdom'] - t['2006']['United Kingdom']
    #raise NotImplementedError()
answer_four()

246702696075.3999

In [119]:
# Cell for autograder.


### Question 5
What is the mean energy supply per capita?

*This function should return a single number.*

In [120]:
def answer_five():
    t = answer_one()
    return np.mean(t['Energy Supply per Capita'])
    #raise NotImplementedError()
answer_five()

157.6

In [121]:
# Cell for autograder.


### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [122]:
def answer_six():
    t = answer_one()
    m = t['% Renewable'].sort_values()
    return (m[m == m[-1]].index[0],round(m[-1]))
    #raise NotImplementedError()
answer_six()

('Brazil', 70)

In [123]:
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"

assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"


### Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [124]:
def answer_seven():
    m = answer_one()
    t = (m['Self-citations']/m['Citations']).dropna()
    t = t.sort_values()
   # raise NotImplementedError()
    return (t[t == t[-1]].index[0],t[-1])
answer_seven()

('China', 0.6893126179389422)

In [125]:
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"

assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"


### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

In [126]:
def answer_eight():
    m = answer_one()
    t = (m['Energy Supply per Capita']/m['Energy Supply']).dropna()
    t = t.sort_values()
    return t[t == t[2]].index[0]
    #raise NotImplementedError()
answer_eight()

'United States'

In [127]:
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"


### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [128]:
def answer_nine():
    import scipy.stats as stats
    import matplotlib.pyplot as plt
    t = answer_one()
    t['PopEst'] = t['Energy Supply'] / t['Energy Supply per Capita']
    t['Citable docs per Capita'] = t['Citable documents'] / t['PopEst']
    #raise NotImplementedError()
    #plt.scatter('Citable docs per Capita','Energy Supply per Capita',data = t)
    m =  t['Citable docs per Capita']
    n = t['Energy Supply per Capita']
    corr, pval =stats.pearsonr(m,n)
    return corr
answer_nine()

0.7940010435442943

In [129]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [130]:
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"


### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [131]:
def answer_ten():
    t = answer_one()
    m = t['% Renewable'].median()
    t.loc[t['% Renewable'] >= m, 'AOB']= 1
    t.loc[t['% Renewable'] < m, 'AOB']= 0
    #raise NotImplementedError()
    return t['AOB']
answer_ten()

Country
China                1.000000
United States        0.000000
Japan                0.000000
United Kingdom       0.000000
Russian Federation   1.000000
Canada               1.000000
Germany              1.000000
India                0.000000
France               1.000000
South Korea          0.000000
Italy                1.000000
Spain                1.000000
Iran                 0.000000
Australia            0.000000
Brazil               1.000000
Name: AOB, dtype: float64

In [132]:
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"


### Question 11
Use the following dictionary to group the Countries by Continent, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [133]:
def numbers_to_strings(m,cd):
    a,b,c,d,e = ([] for i in range(5))
    for i in m:
        if cd[i] =='Asia' : a.append(i)
        elif cd[i] =='North America' : b.append(i),
        elif cd[i] =='Europe' : c.append(i),
        elif cd[i] =='South America' : d.append(i), 
        elif cd[i] =='Australia' : e.append(i)
        else: continue
    return a,b,c,d,e

In [134]:
def answer_eleven():
    from statistics import mean,stdev
    t = answer_one()
    cd=  {'China':'Asia', 'United States':'North America', 'Japan':'Asia', 'United Kingdom':'Europe', 'Russian Federation':'Europe', 'Canada':'North America', 
          'Germany':'Europe', 'India':'Asia','France':'Europe', 'South Korea':'Asia', 'Italy':'Europe', 'Spain':'Europe', 'Iran':'Asia',
          'Australia':'Australia', 'Brazil':'South America'}
    a,b,c,d,e=numbers_to_strings(t.index.values,cd)
    t['PopEst'] = t['Energy Supply'] / t['Energy Supply per Capita']
    y = ['Asia', 'Australia', 'Europe', 'North America', 'South America']
    f= {}
    for i,m in zip( [a,e,c,b,d],y):
        p,q = 0,[]
        for j in i:
            p += t['PopEst'][j]
            q.append(t['PopEst'][j])
        x = ['size', 'sum', 'mean', 'std']
        r = p / len(q)
        if len(q)  == 1:
            s = None
        else:
            s = stdev(q)
        f[m] = {'size':len(q),'sum':p,'mean':r,'std': s}
    g = pd.DataFrame.from_dict(f)
    return g.T
    #raise NotImplementedError()
answer_eleven()

,size,sum,mean,std
Asia,5.000000,2898666386.610600,579733277.322120,679097888.366095
Australia,1.000000,23316017.316017,23316017.316017,nan
Europe,6.000000,457929667.216372,76321611.202729,34647667.065623
North America,2.000000,352855249.480250,176427624.740125,199669644.856668
South America,1.000000,205915254.237288,205915254.237288,nan


In [135]:
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"

assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"

assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"


### Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [269]:
def answer_twelve():
    df = answer_one()
    df["Continent"] = np.nan
    df['Bins'] = pd.cut(df['% Renewable'],5)
    cd=  {'China':'Asia', 'United States':'North America', 'Japan':'Asia', 'United Kingdom':'Europe', 'Russian Federation':'Europe', 'Canada':'North America', 
          'Germany':'Europe', 'India':'Asia','France':'Europe', 'South Korea':'Asia', 'Italy':'Europe', 'Spain':'Europe', 'Iran':'Asia',
          'Australia':'Australia', 'Brazil':'South America'}
    for i in df.index.values:
        if i in cd:
            df['Continent'][i] = cd[i]
    #raise NotImplementedError()
    m =df.groupby([ 'Continent','Bins'  ]).size()
    m= m.rename_axis(['Continent','% Renewable'])
    return m
answer_twelve()

Continent      % Renewable     
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
dtype: int64

In [219]:
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"

assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"


### Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a series `PopEst` whose index is the country name and whose values are the population estimate string*

In [209]:
def thousand_count(m):
    count=0
    n = str(m)
    t= n.split('.')
    u = "{:,.0f}". format(int(t[0]))
    if len(t) == 1:
        return u
    else:
        return (u+'.'+t[1])

In [210]:
def answer_thirteen():
    import locale
    import decimal
    locale.setlocale( locale.LC_ALL, '' )
    t = answer_one()
    n ={}
    t['PopEst'] = t['Energy Supply'] / t['Energy Supply per Capita']
    for i in t.index.values:
        m = t['PopEst'][i]
        if m.is_integer() ==True:
            u = thousand_count(m)
            n[i]= u
        else:
            u = thousand_count(m)
            n[i] = u
    #raise NotImplementedError()
    return pd.Series(n)
answer_thirteen()

China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
dtype: object

In [211]:
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"

assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")